In [1]:
import math
import numpy as np
import nltk
nltk.download('punkt')
import os
import pandas as pd
import random
import re
import stanza

from stanza.utils.datasets.ner.utils import write_dataset
from transform_weight_date import number_to_words, date_to_formats

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [2]:
# Read in synthetic data
df = pd.read_csv('../../data/data/generated/data_230124-172021.csv')

In [3]:
# Use re to replace any instances of "####kg" with "#### kg" where #### is any continuous 
# sequence of numbers and unit is one of those listed below
def separate_weight_unit(row):
    return re.sub(r'([0-9]+)(kgs|kg|lbs|lb|pounds|kilograms)', r"\1 \2", row)

# Function to remove spaces (e.g. "Take 3" -> "Take3")
def remove_spaces(row):
    return row.replace(" ", "")

def add_loc_comma_token(row):
    return row.replace(",", " ,")

In [4]:
# Consider using Stanza sentence tokenizer instead of nltk word_tokenizer? See csv_to_BIO_stanzaTokenize.ipynb file

df['text'] = df['text'].apply(lambda x: separate_weight_unit(x))
df['text_split'] = df['text'].apply(lambda x: x.strip())
df['text_split'] = df['text_split'].apply(lambda x: nltk.word_tokenize(x))

# Preprocess orgs and locations 
df['org_no_space'] = df['organization'].apply(lambda x: remove_spaces(x))
df['loc_no_space'] = df['location'].apply(lambda x: remove_spaces(x))
df['location'] = df['location'].apply(lambda x: add_loc_comma_token(x))

# Assign appropriate types
string_cols = ["item1", "item2", "location"]
df[string_cols] = df[string_cols].astype(str)
df['weight2'] = df['weight2'].astype('Int64')

# Compute variations of date and weight formats
df['date_vars'] = df['date'].apply(lambda x: date_to_formats(x))
df['weight1_text'] = df['weight1'].apply(lambda x: number_to_words(x)[1])
df['weight2_text'] = df['weight2'].apply(lambda x: number_to_words(x)[1] if pd.notnull(x) else str(x))

# Make string columns lowercase for comparisons
lowercase_cols = string_cols + ['organization', 'location', 'org_no_space', 'loc_no_space', 'weight1_text', 'weight2_text']
for i in lowercase_cols:
    df[i] = df[i].apply(lambda x: x.lower())

# df.head(15)
# df.info()
# print(df.iloc[12])
df['date_vars'][2]

['2021-05-25',
 'May 25, 2021',
 'may 25, 2021',
 'May 25th, 2021',
 'may 25th, 2021',
 '25 May 2021',
 '25 may 2021',
 '25th May 2021',
 '25th may 2021',
 'May 25 2021',
 'may 25 2021',
 'May 25th 2021',
 'may 25th 2021',
 '05/25/21',
 '05/25/2021',
 '05-25-2021',
 '25 May 2021',
 '25 may 2021',
 '25th May 2021',
 '25th may 2021']

In [ ]:
# Convert data into list of words with associated 'B - entity', 'I - entity' or 'O'
# Look at other preprocessing steps in read_datasets function in convert_bn_daffodil

units = ["kilograms", "kilogram", "kgs", "kg", "lb", "lbs", "pounds", "pound"]

def assign_entity_types(row):
    words = row['text_split']
    new_tags = []
    prev_item_tag = False

    idx = 0
    while (idx < len(words)):
        loc_length = len(row['location'].split())
        org_length = len(row['organization'].split())
        weight1_text_length = len(row['weight1_text'].split())
        if row['weight2_text'] != None:
            weight2_text_length = len(row['weight2_text'].split())
        else:
            weight2_text_length = -1
        
        # Assign location labels
        # Checks for consecutive word matching for full location name (normalizing all words to lowercase)
        # Does not handle extraneous locations not provided in prompt
        if ((idx < len(words) - loc_length) and 
            [x.lower() for x in words[idx : idx + loc_length]] == row['location'].split()):
            new_tags.append("B-LOC")
            idx += 1
            for i in range(1, loc_length):
                new_tags.append("I-LOC")
                idx += 1
        elif (words[idx].lower() == row['loc_no_space']):
            new_tags.append("B-LOC")
            idx += 1

        # Assign organization labels
        # Checks for consecutive word matching for full location name (normalizing all words to lowercase)
        elif ((idx < len(words) - org_length) and 
            [x.lower() for x in words[idx : idx + org_length]] == (row['organization'].lower().split())):
            new_tags.append("B-ORG")            # idea for later: tag acronyms for Orgs?
            idx += 1                            
            for i in range(1, org_length):
                new_tags.append("I-ORG")
                idx += 1
        elif (words[idx].lower() == row['org_no_space']):
            new_tags.append("B-ORG")      
            idx += 1
            
        # Assign unit labels
        elif any(words[idx] == word for word in units):   
            new_tags.append("B-UNT")
            idx += 1
        
        # Assign weight labels for numeric and text numbers (consider '-' and non- '-' versions of written numbers?)
        elif (words[idx] == str(row['weight1']) or (row['weight2'] != None and words[idx] == str(row['weight2']))): 
            new_tags.append("B-WEI")
            idx += 1
        elif ((idx < len(words) - weight1_text_length) and 
                [x.lower() for x in words[idx : idx + weight1_text_length]] == row['weight1_text'].split()):
            new_tags.append("B-WEI")
            idx += 1
            for i in range(1, weight1_text_length):
                new_tags.append("I-WEI")
                idx += 1
        elif ((weight2_text_length > 0) and (idx < len(words) - weight2_text_length) and 
                [x.lower() for x in words[idx : idx + weight2_text_length]] == row['weight2_text'].split()):
            new_tags.append("B-WEI")
            idx += 1
            for i in range(1, weight1_text_length):
                new_tags.append("I-WEI")
                idx += 1

        # Assign item labels (dont look for consecutive matches here)
        elif (any(words[idx] == word for word in row['item1'].split()) or 
                                (row['item2'] != None and any(words[idx] == word for word in row['item2'].split()))):
            if prev_item_tag: 
                new_tags.append("I-ITM")
            else:
                new_tags.append("B-ITM")
                prev_item_tag = True
            idx += 1
        # Open question: How to assign dates? Need to capture all possible date formats?
        # Solution: convert golden value dates to datetime objects then use strftime package to generate 
        # possible text versions of it
        # Assign date labels
        # elif () UPDATE MEEEEEE
        
        else:
            new_tags.append("O")
            prev_item_tag = False
            idx += 1

    return list(zip(words, new_tags))

df['tagged_entities'] = df.apply(assign_entity_types, axis =1)

In [ ]:
# Test sentences
# df['text_split'][13] = ['I', 'have', 'two', 'hundred', 'and', 'twenty', 'dogs', 'and', 'two', 'hundred', 'and', 'fifty-seven', 
#                         'cats', 'ugly', 'two', 'hundred', 'and', 'twenty-one']
# assign_entity_types(df.iloc[13])

In [ ]:
# Review newly assigned non-"O" tags
SAMPLE_NO = 12
for i in df.iloc[SAMPLE_NO]['tagged_entities']:
    if i[1] != "O":
        print(i[0], i[1])

print(df['tagged_entities'][SAMPLE_NO])
df['text'][SAMPLE_NO]

In [ ]:
# Compile all sentences into a single list of lists (sentences) of word-pairs (word, NER tag)

# Open question: this method is not very robust (cross-references stanza tokenizer sentence lengths 
# against list of original sentence text words, which might not be 1-1).
#   Solution: use a find() to search for first word in each sentence, not just blind indexing into paragraph text

# Method is not efficient. Maybe could be vectorized (?), but we only have to run this script once

nlp = stanza.Pipeline(lang='en', processors='tokenize')
df['text_stanza_tokenize'] = df['text'].apply(lambda x: nlp(x))

all_sentences = []
for i in range(len(df)):
    idx = 0
    for sentence in df.iloc[i]['text_stanza_tokenize'].sentences:
        new_sentence = list(df.iloc[i]['tagged_entities'][idx:idx+len(sentence.words)])
        all_sentences.append(new_sentence)
        idx += len(sentence.words)

# print(all_sentences[10])


In [ ]:
# Divide data into datasets = (train_sentences, dev_sentences, test_sentences)

DEV_SPLIT = 0.1
TEST_SPLIT = 0.1

random.seed(1234)
random.shuffle(all_sentences)

train_sentences = all_sentences[ : int(len(all_sentences)*(1-DEV_SPLIT-TEST_SPLIT))]
dev_sentences = all_sentences[int(len(all_sentences)*(1-DEV_SPLIT-TEST_SPLIT)) : int(len(all_sentences)*(1-TEST_SPLIT))]
test_sentences = all_sentences[int(len(all_sentences)*(1-TEST_SPLIT)) : ]

# print(len(train_sentences))
# print(len(dev_sentences))
# print(len(test_sentences))
# print(len(all_sentences))

datasets = (train_sentences, dev_sentences, test_sentences)

In [ ]:
# Convert file and write to JSON file needed for Stanza modelling
out_directory = os.getcwd()
write_dataset(datasets, out_directory, "TOC_Test")

In [ ]:
# Convert to JSON file needed by Stanza model
# There is a conversion script called several times in prepare_ner_dataset.py which converts IOB format to our internal NER format:
# import stanza.utils.datasets.ner.prepare_ner_file as prepare_ner_file

# prepare_ner_file.process_dataset(input_iob, output_json)